# Régularisation LASSO ($\ell_1$)

## Théorie

### Fondamentaux

La régression LASSO (Least Absolute Shrinkage and Selection Operator) est une technique de régularisation qui ajoute une pénalité L1 aux coefficients du modèle. 

Cette méthode présente deux avantages majeurs : elle permet une sélection automatique des variables en réduisant certains coefficients exactement à zéro, tout en limitant le sur-apprentissage grâce à la pénalisation des coefficients élevés.

Contrairement à la régularisation Ridge (L2) qui ne fait que réduire l'amplitude des coefficients, le LASSO peut éliminer des variables du modèle. C'est pourqoui on l'utilise pour nos modèle de régression linéaire parcimonieux.

Pour ce faire, on résout ce problème de minimisation :

$$ \boxed{\min_{\beta\in\mathbb{R}^p}\frac{1}{2n}\|y-X\beta\|_2^2+\lambda\|\beta\|_1}$$

avec une pénalité $\|\beta\|_1=\sum_{j=1}^p|\beta_j|$

### Choix du $\lambda$

On utilise ici l'algorithme détaillé dans OUTILS/QUT.ipynb spécifique à LASSO en connaissant le paramètre de bruit (oracle)

### Optimisation

Pour résoudre ce problème de minisation on peut utiliser dans notre cas 2 méthodes ISTA et CD (Coordinate descent) qui sont expliqué en profondeur dans les notebooks présent dans le dossier OUTILS.

#### ISTA

On se ramène à un problème à résoudre de cette forme :  
$$\min_{\beta\in\mathbb{R}^p} F(\beta) := f(\beta) + g(\beta)$$

avec

- $f(\beta) = \dfrac{1}{2n} \|y - X\beta\|_2^2$ :  
    Il s'agit du terme d'erreur quadratique (moindre carrés) (MSE).  
    $f$ est une fonction convexe, différentiable, à gradient lipschitzien.

- $g(\beta) = \lambda \|\beta\|_1$ :  
    Il s'agit du terme de pénalisation $\ell_1$. 
    $g$ est convexe, non différentiable, mais proximable.

On peut ainsi très bien utiliser ISTA pour résoudre ce problème.

Le gradient de la fonction $f$ se calcule comme suit :

$$
\nabla f(\beta) = -\frac{1}{n} X^\top (y - X\beta)
$$

##### Constante L


Par définition,  $L$ est tel que :
$$\|\nabla f(a) - \nabla f(b)\|_2 \leq L \|a - b\|_2, \quad \forall a, b \in \mathbb{R}^p$$

Or, 
$$\nabla f(\beta) = -\frac{1}{n} X^\top (y - X\beta)$$

Donc 
$$\|\nabla f(a) - \nabla f(b)\|_2 = \left\| \frac{1}{n} X^\top X (a - b) \right\|_2 \leq \left\| \frac{1}{n} X^\top X \right\|_2 \|a - b\|_2$$

Ainsi, 
$$L = \left\| \frac{1}{n} X^\top X \right\|_2 = \frac{1}{n} \|X\|_2^2$$

##### Proximal de g

Dans notre cas, le proximal du terme de pénalisation $\ell_1$ avec un pas $\gamma = 1/L$ s'écrit :

$$
\text{prox}_{\frac{\lambda}{L} \|\cdot\|_1}(z) = \arg\min_{x \in \mathbb{R}^p} \left\{ \frac{L}{2} \|x - z\|_2^2 + \lambda \|x\|_1 \right\}
$$

Ce proximal correspond toujours à l'opérateur de seuillage doux:

$$
\text{prox}_{\frac{\lambda}{L} \|\cdot\|_1}(z) = \operatorname{sign}(z) \odot \max\left(|z| - \frac{\lambda}{L},\, 0\right)
$$

où $\odot$ désigne le produit terme à terme (Hadamard) entre les vecteurs.

#### Coordinate descent

Comme pour ISTA, on se ramène à un problème de la forme suivante :

$$\min_\beta h(\beta) := f(\beta) + \lambda \Omega(\beta),$$

où dans notre cas :

- $f : \mathbb{R}^p \to \mathbb{R} = \dfrac{1}{2n} \|y - X\beta\|_2^2$
- $\Omega : \mathbb{R}^p \to \mathbb{R}$ est la fonction de pénalisation, non lisse mais séparable :
    - $\Omega(\beta) = \sum_{j=1}^p \Omega_j(\beta_j)$ avec $\Omega_j : \mathbb{R} \to \mathbb{R} = |\beta_j|$.
- $\lambda > 0$ est le paramètre de régularisation.

##### Constante L

C'est la même que pour ista

##### Proximal de g

Dans notre cas, le proximal du terme de pénalisation $\ell_1$ avec un pas $\gamma = 1/L$ s'écrit :

$$
\text{prox}_{\frac{\lambda}{L} \|\cdot\|_1}(v) = \arg\min_{x \in \mathbb{R}^p} \left\{ \frac{L}{2} \|x - v\|_2^2 + \lambda \|x\|_1 \right\}
$$

Ce proximal correspond toujours à l'opérateur de seuillage doux :

$$
\left[\text{prox}_{\frac{\lambda}{L} \|\cdot\|_1}(v)\right]_j = \text{sign}(v_j) \cdot \max(|v_j| - \frac{\lambda}{L},\, 0)
$$

## Simulations

In [7]:
import sys, pathlib
sys.path.append(str(pathlib.Path.cwd().parent.parent.parent.parent)+"\AYMEN\EXPLORATION\CAS LINEAIRE\OUTILS")

### LASSO avec ISTA en ORACLE

In [8]:
from simulations import SimulationLassoOracleIsta

Sim = SimulationLassoOracleIsta(
    n = 25,
    p = 50,
    list_s = [1, 2, 3],
    sigma = 0.0,
    simu_iter=10,
    qut_iter=100,
    max_iter=100,
    verbose=True
)

Sim.run()

Sim.plot()


TypeError: 'numpy.ndarray' object is not callable